## Creating a delta database
First we need a place to store our data. Databricks Unity Catalog organizes data in a three-level namespace or format:
1. Catalog
1. Datbase (aka Schema)
1. Table

This makes it easy for Databricks Lakehouse AI and Unity Governance to function. This also provides an idea of structure to architects without locking anything in too much in terms of design.

In [0]:
# %sql
# --TODO: Update the following with your own catalog and database names
# --SQL is annoying - we'll have to update these manually for now.

# --Update the catalog name with your own
# CREATE CATALOG IF NOT EXISTS ademianczuk;

# --Update the catalog & database name with your own
# CREATE DATABASE IF NOT EXISTS ademianczuk.ncr;

# --Update the catalog & database name with your own (leave `data` alone for this notebook. It's just the name of your volume)
# CREATE VOLUME IF NOT EXISTS ademianczuk.ncr.data

In [0]:
import hashlib, base64

#IMPORTANT! DO NOT CHANGE THESE VALUES!!!!
# catalog = "workshop"
# db = "default"

current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
hash_object = hashlib.sha256(current_user.encode())
hash_user_id = base64.b32encode(hash_object.digest()).decode("utf-8").rstrip("=")[:12]  #Trim to 12 chars for readability
initials = "".join([x[0] for x in current_user.split("@")[0].split(".")])
short_hash = hashlib.md5(current_user.encode()).hexdigest()[:8]  #Short 8-char hash
safe_user_id = f"{initials.upper()}_{short_hash}"

print(safe_user_id)

AD_82257556


In [0]:
#TODO: Update the following with your own catalog and database names
# catalog = "ademianczuk"
# database = "ncr"

catalog = "main"
database = safe_user_id
volume = "data"

In [0]:
# CREATE DATABASE IF NOT EXISTS ademianczuk.ncr;
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog}.{database};")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog}.{database}.{volume}")

DataFrame[]

## Reading in our source data
I've included the original source data in the root of this project repository. We're also going to be reading it from a public URL to show how we can source data from pretty much anywhere. We're going to show examples of both reading from a datbricks volume (that may be mapped to a cloud storage container) as well as the same data from a public URL to give you an idea of different ways to ingest source data.

In [0]:
%sh

#TODO: Update the following with your own catalog and database names
CATALOG=main
DATABASE=AD_82257556
#DATABASE={YOUR_SAFE_USER_ID}

#Create a temp storage location for our downloaded file
rm -rf /tmp/ncr || true
mkdir -p /tmp/ncr
cd /tmp/ncr

#Download & extract the gardening archive
curl -L https://raw.githubusercontent.com/andrijdemianczuk/Ingenius_Impala/refs/heads/main/Original_Data/Wind_Data_2025-v2.csv -o Wind_Data_2025-v2.csv

#Move the dataset to our main bucket. Since we're using the root volume directory, we can't manage it with normal sh commands, but for example we'll show it here for posterity. Downloading the same file again will simply overwrite the old one.

# rm -rf /Volumes/$CATALOG/$DATABASE/data/ || true
# mkdir -p /Volumes/$CATALOG/$DATABASE/data/
cp -f Wind_Data_2025-v2.csv /Volumes/$CATALOG/$DATABASE/data/

rm -rf /Volumes/$CATALOG/$DATABASE/data/csv || true
mkdir -p /Volumes/$CATALOG/$DATABASE/data/csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  341k  100  341k    0     0  1442k      0 --:--:-- --:--:-- --:--:-- 1445k


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import year, month, dayofweek
from pyspark.sql import DataFrame

In [0]:
df = (
    spark.read.option("header", True)
    .option("multiline", True)
    .option("quote", '"')
    .option("escape", '"')
    .option("inferSchema", True)
    .csv(f"/Volumes/{catalog}/{database}/data/Wind_Data_2025-v2.csv")
)

In [0]:
display(
    df.filter((df["FORECAST_DATE_GMT"] >= "2025-01-01") & (df["FORECAST_DATE_GMT"] <= "2025-01-31")).orderBy(
        df["FORECAST_DATE_GMT"].asc()
    )
)

FORECAST_DATE_LOCAL,FORECAST_DATE_GMT,MAX,MIN,OPT,MCR,ACTUAL
2025-01-01T00:00:00.000Z,2025-01-01T07:00:00.000Z,831.53,255.82,562.76,5688,421.66
2025-01-01T01:00:00.000Z,2025-01-01T08:00:00.000Z,833.23,267.56,526.15,5688,360.2
2025-01-01T02:00:00.000Z,2025-01-01T09:00:00.000Z,869.65,270.67,494.45,5688,353.9
2025-01-01T03:00:00.000Z,2025-01-01T10:00:00.000Z,946.79,246.95,475.73,5688,367.46
2025-01-01T04:00:00.000Z,2025-01-01T11:00:00.000Z,1022.32,205.16,482.95,5688,319.72
2025-01-01T05:00:00.000Z,2025-01-01T12:00:00.000Z,944.45,194.13,459.62,5688,224.74
2025-01-01T06:00:00.000Z,2025-01-01T13:00:00.000Z,818.1,201.45,428.76,5688,179.33
2025-01-01T07:00:00.000Z,2025-01-01T14:00:00.000Z,709.83,200.39,413.06,5688,142.51
2025-01-01T08:00:00.000Z,2025-01-01T15:00:00.000Z,721.42,199.19,404.38,5688,121.75
2025-01-01T09:00:00.000Z,2025-01-01T16:00:00.000Z,655.73,184.09,372.63,5688,89.31


In [0]:
df.count()

5039

In [0]:
display(df.agg(min("FORECAST_DATE_GMT"), max("FORECAST_DATE_GMT")))

min(FORECAST_DATE_GMT),max(FORECAST_DATE_GMT)
2025-01-01T07:00:00.000Z,2025-07-30T05:00:00.000Z


## Organizing our data
Let's break up the dataframe by month (or by week) and write out the parquet files accordingly. For the lab, we'll be separating our data into a number of files so that we can 'drop' them in to our system as a simulation of real data coming in to some type of cloud storage (e.g., ADLS, S3 or GCS buckets). We'll be using Databricks Volumes for this lab. Databricks recommends mapping these Volumes to your cloud storage container whenever possible, however it's not necessary and you can connect to your storage connectors externally if you prefer.

In [0]:


df = df.withColumn('dayOfWeek', dayofweek(col('FORECAST_DATE_GMT')))
df = df.withColumn('dayOfMonth', dayofmonth(col('FORECAST_DATE_GMT')))
df = df.withColumn('month', month(col('FORECAST_DATE_GMT')))
df = df.withColumn('year', year(col('FORECAST_DATE_GMT')))

In [0]:
display(df)

FORECAST_DATE_LOCAL,FORECAST_DATE_GMT,MAX,MIN,OPT,MCR,ACTUAL,dayOfWeek,dayOfMonth,month,year
2025-01-01T00:00:00.000Z,2025-01-01T07:00:00.000Z,831.53,255.82,562.76,5688,421.66,4,1,1,2025
2025-01-01T01:00:00.000Z,2025-01-01T08:00:00.000Z,833.23,267.56,526.15,5688,360.2,4,1,1,2025
2025-01-01T02:00:00.000Z,2025-01-01T09:00:00.000Z,869.65,270.67,494.45,5688,353.9,4,1,1,2025
2025-01-01T03:00:00.000Z,2025-01-01T10:00:00.000Z,946.79,246.95,475.73,5688,367.46,4,1,1,2025
2025-01-01T04:00:00.000Z,2025-01-01T11:00:00.000Z,1022.32,205.16,482.95,5688,319.72,4,1,1,2025
2025-01-01T05:00:00.000Z,2025-01-01T12:00:00.000Z,944.45,194.13,459.62,5688,224.74,4,1,1,2025
2025-01-01T06:00:00.000Z,2025-01-01T13:00:00.000Z,818.1,201.45,428.76,5688,179.33,4,1,1,2025
2025-01-01T07:00:00.000Z,2025-01-01T14:00:00.000Z,709.83,200.39,413.06,5688,142.51,4,1,1,2025
2025-01-01T08:00:00.000Z,2025-01-01T15:00:00.000Z,721.42,199.19,404.38,5688,121.75,4,1,1,2025
2025-01-01T09:00:00.000Z,2025-01-01T16:00:00.000Z,655.73,184.09,372.63,5688,89.31,4,1,1,2025


## Why write to parquet?
Parquet is a storage and performance efficient file format. Since it is columnar in nature, it is considered to be a 'dense' storage format. Parquet also allows for data partitioning making it very fast and efficient for filtering data when reading in (since partitions are stored in directories, parquet files that aren't part of the search terms can be ignored). This concept is important to understanding how Delta works as well. Delta is very similar to parquet but also includes metadata at the parent directory level about how data is stored and organized.

Although we don't need to write to parquet here, it's good to show an example of how it works. In reality, we'd just read in the raw data to a dataframe or some type of materialization and start working with our data from that point on. Another thing that parquet allows us to do is 'pick up' our workload from this point on in the notebook if we terminate our cluster. Think of it as a 'save state' for our work. Delta works even better for this, so we'll be showing both examples below.

In [0]:
df.write.partitionBy("year", "month").mode("overwrite").format("parquet").save(f"/Volumes/{catalog}/{database}/{volume}/Wind_Data_2025-v2")

In [0]:
df = spark.read.parquet(f"/Volumes/{catalog}/{database}/data/Wind_Data_2025-v2/year=2025/month=1")
display(df)
df.count()

FORECAST_DATE_LOCAL,FORECAST_DATE_GMT,MAX,MIN,OPT,MCR,ACTUAL,dayOfWeek,dayOfMonth
2025-01-31T16:00:00.000Z,2025-01-31T23:00:00.000Z,3494.17,924.09,3113.59,5688,2681.7,6,31
2025-01-31T15:00:00.000Z,2025-01-31T22:00:00.000Z,3452.52,1103.94,3054.91,5688,2728.64,6,31
2025-01-31T14:00:00.000Z,2025-01-31T21:00:00.000Z,3375.78,1388.9,2955.75,5688,2716.7,6,31
2025-01-31T13:00:00.000Z,2025-01-31T20:00:00.000Z,3433.98,1377.02,2786.87,5688,2789.84,6,31
2025-01-31T12:00:00.000Z,2025-01-31T19:00:00.000Z,3317.78,1328.04,2537.91,5688,2691.87,6,31
2025-01-31T11:00:00.000Z,2025-01-31T18:00:00.000Z,3019.64,1315.04,2150.82,5688,2469.89,6,31
2025-01-31T10:00:00.000Z,2025-01-31T17:00:00.000Z,2701.77,1360.15,2032.61,5688,2203.52,6,31
2025-01-31T09:00:00.000Z,2025-01-31T16:00:00.000Z,2552.43,1126.9,1944.35,5688,2006.93,6,31
2025-01-31T08:00:00.000Z,2025-01-31T15:00:00.000Z,2489.79,1250.53,1858.66,5688,1930.84,6,31
2025-01-31T07:00:00.000Z,2025-01-31T14:00:00.000Z,2406.5,1350.91,1778.8,5688,1712.51,6,31


737

In [0]:
df = spark.read.parquet(f"/Volumes/{catalog}/{database}/data/Wind_Data_2025-v2")
df.count()

5039

## Creating a number of files to simulate incremental ingestion
Now that we have everything organized, let's split up our dataframe into a csv each representing a month's worth of data. We will incrementally load each file in the pipeline later on and will give us an idea of how merging works along with in-flight etl.

In [0]:
base_dir = f"/Volumes/{catalog}/{database}/{volume}/csv"

#Get distinct months (assumes 'month' is 1..12; if it's yyyy-MM use that string directly)
months = [r.month for r in df.select("month").distinct().collect()]

def write_month_csv(src_df: DataFrame, month_val):
    
    #zero-pad to 2 digits if month is numeric; adjust to your format
    m_str = f"{int(month_val):02d}" if isinstance(month_val, (int,)) else str(month_val)
    tmp_dir = f"{base_dir}/_tmp_month_{m_str}"

    #1. write to a temp directory with a single part file
    (src_df
        .filter(col("month") == month_val)
        .coalesce(1)
        .write
        .mode("overwrite")
        .option("header", "true")
        .option("mapreduce.fileoutputcommitter.marksuccessfuljobs","false")
        .csv(tmp_dir))

    #2. find the single part file
    part_files = [f.path for f in dbutils.fs.ls(tmp_dir) if f.name.startswith("part-") and f.name.endswith(".csv")]
    if not part_files:
        raise RuntimeError(f"No CSV part file found for month {m_str} in {tmp_dir}")
    part_path = part_files[0]

    # 3. move/rename to final destination (flat structure)
    final_path = f"{base_dir}/month_{m_str}.csv"
    dbutils.fs.mv(part_path, final_path, True)

    #4. clean up the temp directory
    dbutils.fs.rm(tmp_dir, True)

for m in months:
    write_month_csv(df, m)


## Next Steps
Now that we have our data all ready to go, we'll create a small application that simulates dropping each file into another Databricks volume for ingestion. This volume will simulate being attached to an external storage container such as ADLS, GCS or S3.